In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Walks for Finance Part 1: Introduction to Discrete Time Quantum Walks
$
\renewcommand{\ket}[1]{|{#1}\rangle}
\renewcommand{\bra}[1]{\langle{#1}|}
$

--- 
## Motivation
This tutorial serves as an introduction to quantum walks, using the discrete time quantum walk as an example. It prepares you for the next tutorial, where we will apply quantum walks to the following finance-inspired problem:



> **Encoding Probability Distributions Problem**:  Given a discrete probability distribution $\mathcal{P}$, generate a quantum kernel (i.e., a sequence of quantum gates) that transforms an initial state into a final state whose measurement outcomes match the target distribution $\mathcal{P}$.

One approach to tackle this problem is through random walks by using a variational algorithm to search for the random walk that will generate a targeted distribution, as depicted in the animation below. 

<img src="https://raw.githubusercontent.com/NVIDIA/cuda-q-academic/refs/heads/main/images/quantum_walk_target.gif" width="600">






First let's install the necessary packages.

In [ ]:
## Instructions for Google Colab. You can ignore this cell if you have cuda-q set up and have 
# all the dependent files on your system
# Run this notebook in a CPU runtime
# Uncomment the lines below and execute the cell to install cuda-q

#!pip install cudaq

#!wget -q https://github.com/nvidia/cuda-q-academic/archive/refs/heads/main.zip
#!unzip -q main.zip
#!mv cuda-q-academic-main/quantum-applications-to-finance/images ./images

In [ ]:
import cudaq
import numpy as np
import matplotlib.pyplot as plt
from helper_functions import *

In [ ]:
# Helper function

def plot_results(result, num_qubits):
    # Define a dictionary of results 

    # Initialize the dictionary with all possible bit strings of length 4 for the x axis
    result_dictionary = {}

    # Generate all possible bit strings of length num_qubits
    for i in range(2**num_qubits):
        bitstr = bin(i)[2:].zfill(num_qubits)
        result_dictionary[bitstr] = 0

    # Update the results dictionary of results from the circuit sampling
    for k,v in result.items():
        result_dictionary[k] = v

    # Convert the dictionary to lists for x and y values
    x = list(result_dictionary.keys())

    y = list(result_dictionary.values())

    # Create the histogram
    plt.bar(x, y, color='#76B900')

    # Add title and labels
    plt.title("Sampling of the DTQW")
    plt.xlabel("Positions")
    plt.ylabel("Frequency")

    # Rotate x-axis labels for readability
    plt.xticks(rotation=45)

    # Show the plot
    plt.tight_layout()
    plt.show()

plot_results(result, num_qubits)

---
## Comparing Classical Random Walks with Discrete Time Quantum Walks 

Let's consider a discrete walk taking place on a line. In this scenario, we envision a walker progressing along the 
x-axis in discrete increments, with the movement governed by predefined rules based on coin flips. 


A classical random walk can be viewed as a decision tree, as in the diagram below.  Here, a walker begins in the center of the line. After a coin flip, the walker moves to the left or right depending on the outcome of the coin flip (e.g., flipping heads will send the walker one step to the left, while flipping tails will send the walker one step to the right).  After $n$ coin flips, we record the position of the walker.  This experiment is repeated multiple times to generate a histogram of the walker's final position.

<img src="images/classical-walk.png" width="600">


Let's see this in action with the random walk widget below. The green dot represents the walker whose movements are dictated by flips of a coin whose fairness is controlled by the probability slider. The ending position from each run of the experiment is recorded in the histogram.  Experiment by changing the number of total steps, the probabilities, and the rules for taking a step. What patterns do you notice?

> **Note:** If the widget does not appear below, you can access it directly using [this link](https://nvidia.github.io/cuda-q-academic/quantum-applications-to-finance/images/ss-random-walk.html).

<iframe src="https://nvidia.github.io/cuda-q-academic/quantum-applications-to-finance/images/ss-random-walk.html" width="800" height="600"></iframe>

### Quantum Walks
Unlike classical random walks, where probabilities dictate movement, **quantum walks rely on amplitudes**. Interference between paths leads to a faster spread over the position space, making quantum walks useful for generating more complex probability distributions.  


A **discrete-time quantum walk (DTQW)** is the quantum analogue of a classical random walk. In this scenario, both the position of the walk and the coin are represented as quantum states, $\ket{\psi_W}$ and $\ket{\psi_C}$, respectively. 

Throughout this walk, the state of the walker changes according to quantum shift operators ($S_+, S_-$) which depend on the state of the coin $\ket{\psi_C}$. In particular, the $S_{-}$ operation checks if $\ket{\psi_C}$ in the state $\ket{0}$ (i.e., heads) and if so, induces a shift left operation $DEC$ on the quantum walker. Similarly, $S_{+}$ is defined for tails and shifting right. At each time step, the state of the coin changes (i.e., is "flipped") via a quantum operation ($F$), which we'll call the coin-flip operator. Unlike the classical random walk, we will not know the position of the walker until the end of the process, when the quantum state of the walker is measured. 

In the diagram below, we illustrate a 2-step quantum walk where the coin-flip operator $F$ is the Hadamard gate.  Notice how the final distribution of the walker's possible positions differs from the classical case.


<img src="images/dtqw-superposition-coin.png" width="600">


#### The Line that the Walker Traverses
 For our example, we consider a line that contains $16$ positions: $0,1,\cdots,15$ encoded with the computational basis states on $4$ qubits: $\ket{\bf{0}} = \ket{0000}, \ket{\bf{1}} = \ket{0001}, \ket{\bf{2}} = \ket{0010}, \cdots, \ket{\bf{15}} = \ket{1111}$.  In other words, the computational basis state $\ket{\bf{i}}$ represents the $i^{th}$ location on the line. 


#### The Walker's Position State
The walker's position after $t$ time steps is given by a linear combination of the computational basis states $$\ket{\psi_W} = \sum_{i}\alpha_i(t)\ket{\bf{i}},$$ for some $\alpha_i(t)\in\mathbb{C}$ satisfying $\sum_{i}|\alpha_i(t)|^2 = 1$.  For instance, if the walker began the experiment at position $0$, then the state of the walker at time $0$ would be $\ket{\psi_W(0)}=\ket{0000}$.  

More interestingly, the walker might begin the experiment in a superposition of two or more positions, as depicted in the figure below.  In this case, the walker's initial state is $\ket{\psi_W(0)}=\frac{1}{\sqrt{2}}\ket{0010}+\frac{1}{\sqrt{2}}\ket{0011}$. The aim of this section is to encode into CUDA-Q the first step of the discrete time quantum walk drawn below.

<img src="images/dtqw-superposition-walker.png" width="600">

### Programming a DTQW with CUDA-Q

Let's start coding up one step of the DTQW in CUDA-Q. We will model our program on the image above. 

> **🔧 Exercise ** The first step is to edit the code block below to create a kernel that generates the walker's initial position state: $\ket{\psi_W(0)}=\frac{1}{\sqrt{2}}\ket{0010}+\frac{1}{\sqrt{2}}\ket{0011}$ 
>
> Think about the gates you've learned that can manipulate a qubit's state and how linearity applies. remember that the qubits are indexed as |q[0],q[1],q[2],q[3]>



In [ ]:
# EXERCISE 
# Define a kernel that prepares the walker qubits in an
# equal superposition of the states |2> = |0010> and |3> = |0011>.

@cudaq.kernel
def initial_position(qubits : cudaq.qvector):
    """ Apply gates to the qubits to prepare the inital state of the walker.
    Parameters
        qubits: cudaq.qvector 
                qubits for the walker, (all qubits begin in state |0>)
    """
    
    # Edit the code below this line


    # Edit the code above this line

We can verify that our solution is correct by using the `cudaq.get_state` and _`.amplitudes` commands.  Additionally, in the code block below, we will create a helper function to graph the distribution of the walker's positions, as represented by the state.

In [ ]:
num_qubits = 4

@cudaq.kernel
def check_walker(num_qubits : int):
    """ Kernel to initialize the state of the walker
    Parameters
        num_qubits : int
        Number of qubits for the walker
    """
    # Initialize the qubits
    qubits = cudaq.qvector(num_qubits)
    
    # Apply the initial state to the qubits
    initial_position(qubits)
    
# Get the state of the walker after applying the quantum kernel
state = cudaq.get_state(check_walker, 4)

# Return the amplitudes of |0010> and |0011>
state_amplitudes = state.amplitudes([[0,0,1,0], [0,0,1,1]])

# Print 
precision = 4
print('Walker state array of coefficients:', np.round(np.array(state), precision))
print('Walker statevector: {} |0010> + {} |0011>'.format(np.round(state_amplitudes[0],precision), np.round(state_amplitudes[1],precision)))

# Plot results 
results = cudaq.sample(check_walker, 4, shots_count =1000)
plot_results(results, num_qubits)

## DTQW: one step at a time

In [ ]:
# Set the number of qubits
num_qubits = 4

@cudaq.kernel
def DTQW_one_step(num_qubits: int):

    # INITIALIZATION
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qubit()

    # Initial walker state 1/(sqrt{2}) ( |2>+|3>)
    initial_position(walker_qubits)

    # Initial coin state
    ## place the coin in the |+> state to start.
    # EDIT CODE BELOW THIS POINT

    # EDIT CODE ABOVE THIS POINT

    # One quantum walk step
    ## Flip the coin: The F operation will be a Hadamard gate. 
    # EDIT CODE BELOW THIS POINT


    # EDIT CODE ABOVE THIS POINT

#####     STOP HERE    #########

    # Walker's position change
    ## Shifting right

    ## Shift right when the coin is |1> 

    ## Shifting left

    ## Shift left when the coin is |0>


    # Measure the state of the walker qubits



### Moving the Walker: Controlled Operations
To connect the coin's state to the walker's movement, we use a **controlled gate**. A controlled gate is an operation that acts on one set of qubits (the *target*, our walker) only if another qubit (the *control*, our coin) is in a specific state.

The operation that moves the walker in a discrete time quantum walk works as follows:
* **If the coin qubit is in state $\ket{1}$ (tails),** apply an operator (INC) that shifts the walker one step to the right.
* **If the coin qubit is in state $\ket{0}$ (heads),** apply an operator (DEC) that shifts the walker one step to the left.


The diagram below illustrates the quantum circuit for a single step of the walk. Notice the control symbols on the coin's wire: the filled-in circle (⚫) is standard notation for a control on the $\ket{1}$ state (triggering the INC operation), and the open circle  (⚪) is for a control on the $\ket{0}$ state (triggering the DEC operation).

<img src="https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/images/dtqw-one-step-diagram.png?raw=1" width="600">

Because our quantum coin can be in a superposition like $\frac{1}{\sqrt{2}}(\ket{0} + \ket{1})$, this controlled operation causes the walker to *simultaneously* move left and right. The walker's position evolves into a superposition of being one step to the left and one step to the right of its previous position(s). This is the source of the quantum walk's power to explore many paths at once.


#### Walking the Line

The next step is to define incrementer (`INC`) and decrementer (`DEC`) operations, which will change the state of the walker with shifts to the left and right on the number line, respectively.  

We'll use these operations to build up the shift operators $S_-$ and $S_+$ which depend on the state of the coin.  

These operators can either be defined as custom unitaries using the `cudaq.register_operation`. For instance, `INC` when applied to a basis state $\ket{x}$, the result is $\ket{(x+1)_{\mod{16}}}$ for $x\in \{0,\cdots 15\}$. The unitary matrix below carries out this transformation is:
$$
\begin{pmatrix} 0 & 0 & 0 & 0 & \cdots  \\ 
                        1 & 0 & 0 & 0 & \cdots \\
                        0 & 1 & 0 & 0 & \cdots \\
                        0 & 0 & 1 & 0 & \cdots \\
                        \vdots & \vdots & \vdots & \vdots & \ddots \end{pmatrix} 
$$
Alternatively, we can use use `x.ctrl` gates to implement modular addition, you will see this in the next lab.


In [ ]:
# Define a custom operation on 4 qubits for the INC unitary matrix that
# maps |x> to |x+1> mod 16 and verify that it works as expected for |0000>

num_qubits = 4

# Define the incrementer matrix
def incrementer(num_qubits):
    size = 2**num_qubits
    inc_matrix = np.zeros((size, size))
    for i in range(size):
        inc_matrix[i, (i - 1) % size] = 1
    return inc_matrix

# Create a custom register operation for the incrementer
cudaq.register_operation("INC", incrementer(num_qubits))

@cudaq.kernel
def check_incrementer_kernel():
    qubits = cudaq.qvector(4)
    INC(qubits[0], qubits[1], qubits[2], qubits[3])

# Print the INC circuit
print(cudaq.draw(check_incrementer_kernel))

result = cudaq.sample(check_incrementer_kernel, shots_count=1000).most_probable()
print('Incrementer kernel |0000> -> |{}>'.format(result))


In [ ]:
# Define a kernel on 4 qubits for the DEC operation that
# maps |x> to |x-1> mod 16 and verify that it works as expected for |0001>

# Define the decrementer matrix

def decrementer(num_qubits):
    size = 2**num_qubits
    dec_matrix = np.zeros((size, size))
    for i in range(size):
        dec_matrix[i, (i + 1) % size] = 1
    return dec_matrix


cudaq.register_operation("DEC", decrementer(num_qubits))

# Create a kernel that applies the DEC to the 4-qubit state |0001>
@cudaq.kernel
def check_decrementer_kernel():
    qubits = cudaq.qvector(4)
    # Initialize the qubits to |0001>
    x(qubits[3])
    # Apply the decrementer operation 
    DEC(qubits[0], qubits[1], qubits[2], qubits[3])

result = cudaq.sample(check_decrementer_kernel, shots_count=1000).most_probable()
print('Decrementer kernel |0001> -> |{}>'.format(result))

#### Changing the State of the Walker Based on Coin Flips:

Now, let's put it all together to program one step of the DTQW, using the initial state $\ket{\psi_W(0)} = \frac{1}{\sqrt{2}}(\ket{0010}+\ket{0011})$.

The INC operation below is correctly controlled by the $\ket{1}$ state of the coin using the `ctrl` modifier.

> **🔧 Exercise ** How would you modify the kernel to also include the DEC operation, ensuring it is only triggered when the coin is in the $\ket{0}$ state?
>
> Think about the gates you've learned that can manipulate a qubit's state. And remember, the coin's state is only used to control the walker's movement; its own state shouldn't be permanently altered by the process. How do you ensure the coin is returned to its original state after the DEC operation is applied?

In [ ]:
# EXERCISE
# Complete the step operation
# Edit the code to apply the DEC operation to the walker qubits
# when the coin qubit is in the |0> state

# Set the number of qubits
num_qubits = 4

@cudaq.kernel
def DTQW_one_step(num_qubits: int):
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qubit()

    # Initial walker state 1/(sqrt{2}) ( |2>+|3>)
    initial_position(walker_qubits)

    # Initial coin state
    h(coin_qubit) #CHANGE_ME

    # One quantum walk step
    # Coin operation
    h(coin_qubit) #CHANGE_ME

    # Walker's position change

    ## Shifting right

    ## Shift right when the coin is |1>
    INC.ctrl(coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3])

    ## Shifting left
    ## Shift left when the coin is |0>
    ## EDIT CODE BELOW THIS LINE
    x(coin_qubit)
    DEC.ctrl(coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3])
    x(coin_qubit)

    ## EDIT CODE ABOVE THIS LINE

    # Measure the state of the walker qubits
    mz(walker_qubits)

# Visualize the kernel for the quantum walk
#print(cudaq.draw(DTQW_one_step, num_qubits))

# Sample the kernel for the quantum walk
result = cudaq.sample(DTQW_one_step, num_qubits, shots_count=1000)
print(result)

plot_results(result, num_qubits)

### Avoiding Walking in Circles (optional)

It is important to note that, as currently outlined, we are on track to define a quantum walk that will allow a walker to move between positions $\ket{0}=\ket{0000}$ and $\ket{15}=\ket{1111}$ in one step.  This behavior might not be desired, for instance, in a financial application where the positions of the walker represent stock prices and movement is related to positive or negative investor sentiment. 

To prevent one-step transitions between $\ket{0}$ and $\ket{1}$ , we use an auxiliary qubit (`endpoint_qubit`), Toffoli gates, CNOT gates, and a mid-circuit `reset` to ensure a coin in the state $\ket{1}$ does not trigger a shift right (`INC`) when the walker is at $\ket{1111}$, and a coin in the state $\ket{0}$ does not trigger a shift left (`DEC`) when the walker is in the state $\ket{0000}$. This process is depicted below for one step of the DTQW:

<img src="images/DTQW_endpoint.png" width="600">

The corresponding functions to prevent one-step transitions between $\ket{0000}$ and $\ket{1111}$ are defined below.

In [ ]:
# EXERCISE 
# Define kernels to prevent transitions between |0000> and |1111>

# Kernel to change a coin from 1 to 0 if the walker is in the state |1111>
@cudaq.kernel
def no_INC_at_right_endpoint(walker_qubits : cudaq.qvector, coin_qubit : cudaq.qubit, right_endpoint: cudaq.qubit):
    
    ## EDIT CODE BELOW THIS POINT##
    # Test if the coin is in |1> and the walker state is |1111>,  if so, change the end_point qubit to 1. 
    # Hint: this can be done in one line
    x.ctrl([coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3]], right_endpoint)
    
    # Flip the state of the coin if the endpoint is triggered. Hint, this can be done in one line.
    x.ctrl(right_endpoint, coin_qubit)
    ## EDIT CODE ABOVE THIS POINT##


# Kernel to change a coin from 0 to 1 if the walker is in the state |0000>
@cudaq.kernel
def no_DEC_at_left_endpoint(walker_qubits : cudaq.qvector, coin_qubit : cudaq.qubit, left_endpoint: cudaq.qubit):
    # Bit Flip the walker and coin qubits
    x(coin_qubit)
    x(walker_qubits)
    
    # Trigger the left_endpoint qubit if the walker and the coin are now in the states |1> and |1111> respectively 
    x.ctrl([coin_qubit, walker_qubits[0], walker_qubits[1], walker_qubits[2], walker_qubits[3]], left_endpoint)
    
    # Undo the bit flip on the walker and coin qubits
    x(walker_qubits)
    x(coin_qubit)
    
    # Flip the coin fron |0> to |1> if the endpoint qubit is triggered
    x.ctrl(left_endpoint, coin_qubit)
   
# Kernel to reset the coin and endpoint qubit
@cudaq.kernel()
def reset_coin_and_endpoint(coin_qubit : cudaq.qubit, endpoint: cudaq.qubit):
    # change the coin qubit back if it was targeted by the endpoint qubit
    x.ctrl(endpoint, coin_qubit)

    # reset the endpoint qubit to |0>
    reset(endpoint)